<a href="https://colab.research.google.com/github/RDGopal/IB9AU-2026/blob/main/RAG_1_Long_Context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Long Context vs. RAG
We will first explore an alternative to traditional RAG (Retrieval Augmented Generation). Instead of chopping documents into small pieces ("chunking") and storing them in a database, we will leverage Gemini's Long Context Window.


**Why this matters:** Traditional RAG is like using a search engine to find a single page in a book. It’s fast but can miss context. Long Context is like asking an expert to read the entire book before answering.

**The Advantage:** For "small" datasets (under ~1,000 pages), this method often yields higher accuracy because the model sees the entire document structure (tables, footnotes, and headers) intact.

**The Technique:** We will use Prompt Engineering to force the model into an "Auditor Mode," requiring it to cite specific sections for every claim it makes.

**The Scenario:** You are building a compliance tool for New Horizon Solutions. You need to audit employee records against company policy (Benefits & FMLA) to ensure no rules were broken.


In [ ]:
import google.generativeai as genai
import time
import os

In [ ]:
# 1. Setup (Use the standard library)
# Get your API key from https://aistudio.google.com/

from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
# 2. Upload Files
print("Uploading files...")
# Ensure 'Benefits.pdf' and 'FMLA.pdf' are in your Colab files folder
benefit_file = genai.upload_file("Benefits.pdf")
fmla_file = genai.upload_file("FMLA.pdf")

# 3. Wait for Processing (Crucial Step)
print("Waiting for processing...")
while benefit_file.state.name == "PROCESSING" or fmla_file.state.name == "PROCESSING":
    print(".", end="", flush=True)
    time.sleep(2)
    benefit_file = genai.get_file(benefit_file.name)
    fmla_file = genai.get_file(fmla_file.name)

print("\nFiles are ready.")

In [ ]:
print("Listing available models...")
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

# Answer the question

In [ ]:
# 4. Ask the Question
# We pass the file objects DIRECTLY to the model.
# This uses the 1-Million token context window (Long Context).
model = genai.GenerativeModel("gemini-flash-latest")

response = model.generate_content(
    [
        "You are an HR assistant. Answer based on these documents.",
        benefit_file,
        fmla_file,
        "What is the deductible for Dental? Also, who approved John Smith's FMLA request?"
    ]
)

print("\n--- ANSWER ---")
print(response.text)

#Answer the question with sources


**The Prompt (auditor_prompt):** how you ask is as important as what you ask. By demanding an "Exact Quote", you force the model to look at the pixels of the PDF text, which significantly reduces hallucinations.

**The Validation:** This simple if/else block demonstrates the logic of a Guardrail. In a real FinTech app, if the model fails to output "Source:", the system would automatically flag the answer for human review rather than showing it to a customer.

In [ ]:
# 4. DEFINE THE "AUDITOR" PROMPT
# We don't just ask the question. We give strict instructions on HOW to answer.
# This is "System Prompting" 101 for Finance.
auditor_prompt = """
You are a strict Compliance Auditor for a bank.
Your job is to answer the user's question using ONLY the provided documents.

CRITICAL INSTRUCTION:
For every claim you make, you must provide a "Source Block" in this exact format:
[Source: Document Name | Page/Section | Exact Quote]

Question:
"What is the deductible for Dental and Vision? Also, who approved John Smith's FMLA request?"
"""

# 5. GENERATE CONTENT
# Changing the model to gemini-flash-latest as gemini-1.5-flash was not found.
model = genai.GenerativeModel("gemini-flash-latest")

print("\nAnalyze documents...")
response = model.generate_content(
    [auditor_prompt, benefit_file, fmla_file]
)

# 6. DISPLAY RESPONSE WITH VALIDATION
print("-" * 50)
print("🤖 AI RESPONSE:")
print("-" * 50)
print(response.text)
print("-" * 50)

# 7. (OPTIONAL) VALIDATION CHECK
# In a real app, you would parse the string "[Source: ...]" to build links.
if "Source:" in response.text:
    print("✅ CITATION CHECK: PASSED")
    print("The model provided sources for verification.")
else:
    print("⚠️ CITATION CHECK: FAILED")
    print("Warning: The model did not cite its sources. Human review required.")